In [2]:
import pandas as pd
import numpy as np
import json
from random import randint

In [3]:
FILE_NAME = "2018-07-Domestic-Exchange-Index"

In [4]:
csv_data = pd.read_csv(FILE_NAME+".csv")
csv_data

,ASN-source,ASN,Name,Type,Bandwidth,Gb/s,Connectivity Type
0,AS4652,AS45265,CSL-IX,NIX,50.00,Gbps,IPv6 Dual Stack
1,AS4652,AS38081,TIG-IX,NIX,10.00,Gbps,IPv4
2,AS4652,AS45667,TCCT-IX,NIX,1.00,Gbps,IPv6 Dual Stack
3,AS4652,AS45642,Jastel-IX,NIX,20.00,Gbps,IPv4
4,AS4652,AS45458,AWN-IX,NIX,30.00,Gbps,IPv4
5,AS4652,AS132280,Symphony-ISP,ISP-Local,10.00,Gbps,IPv6 Dual Stack
6,AS4652,AS9931,CAT-ISP,ISP-Local,450.00,Gbps,IPv6 Dual Stack
7,AS4652,AS4618,INET,ISP-Local,31.00,Gbps,IPv4
8,AS4652,AS4765,Pacific Internet,ISP-Local,11.00,Gbps,IPv4
9,AS4652,AS7616,Ji-NET,ISP-Local,1.00,Gbps,IPv4


In [5]:
NODES = set(csv_data["ASN-source"].values)
NODES

{'AS132880',
 'AS133543',
 'AS37930',
 'AS38081',
 'AS45265',
 'AS45458',
 'AS45642',
 'AS45667',
 'AS4652',
 'AS63529'}

In [6]:
EDGES = [(item[0], item[1]) for item in csv_data[["ASN-source", "ASN"]].values]
EDGES

[('AS4652', 'AS45265'),
 ('AS4652', 'AS38081'),
 ('AS4652', 'AS45667'),
 ('AS4652', 'AS45642'),
 ('AS4652', 'AS45458'),
 ('AS4652', 'AS132280'),
 ('AS4652', 'AS9931'),
 ('AS4652', 'AS4618'),
 ('AS4652', 'AS4765'),
 ('AS4652', 'AS7616'),
 ('AS4652', 'AS4741'),
 ('AS4652', 'AS45758'),
 ('AS4652', 'AS24187'),
 ('AS4652', 'AS23932'),
 ('AS4652', 'AS7654'),
 ('AS4652', 'AS24299'),
 ('AS4652', 'AS23884'),
 ('AS4652', 'AS7470'),
 ('AS4652', 'AS4637'),
 ('AS4652', 'AS7693'),
 ('AS4652', 'AS23969'),
 ('AS4652', 'AS38794'),
 ('AS4652', 'AS38888'),
 ('AS4652', 'AS45142'),
 ('AS4652', 'AS45328'),
 ('AS4652', 'AS45606'),
 ('AS4652', 'AS38566'),
 ('AS4652', 'AS131293'),
 ('AS4652', 'AS131090'),
 ('AS4652', 'AS45413'),
 ('AS4652', 'AS5639'),
 ('AS4652', 'AS17887'),
 ('AS4652', 'AS59238'),
 ('AS4652', 'AS4621'),
 ('AS4652', 'AS38328'),
 ('AS4652', 'AS56067'),
 ('AS4652', 'AS134509'),
 ('AS45265', 'AS45642'),
 ('AS45265', 'AS45458'),
 ('AS45265', 'AS38081'),
 ('AS45265', 'AS37930'),
 ('AS45265', 'AS465

In [7]:
def new_node(color, label, attributes, x, y, _id):
    node = {}
    node["color"] = color
    node["label"] = label
    node["attributes"] = attributes
    node["y"] = y
    node["x"] = x
    node["id"] = _id
    node["size"] = len([src for src, dst in EDGES if src == node["id"]])
    return node

N = new_node(color="4f19c7", label="AS4652", attributes={}, x=-739.36383, y=-404.26147, _id="AS4652")
N

{'attributes': {},
 'color': '4f19c7',
 'id': 'AS4652',
 'label': 'AS4652',
 'size': 37,
 'x': -739.36383,
 'y': -404.26147}

In [8]:
def new_edge(sourceID, attributes, targetID, size):
    edge = {}
    edge["sourceID"] = sourceID
    edge["attributes"] = attributes
    edge["targetID"] = targetID
    edge["size"] = size
    return edge
    
E = new_edge("AS4652", {}, "AS45265", 1)
E

{'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS45265'}

In [9]:
colors = [
"#C91F37",
"#DC3023",
"#9D2933",
"#CF000F",
"#E68364",
"#F22613",
"#CF3A24",
"#C3272B",
"#8F1D21",
"#D24D57",
"#F08F907",
"#F47983",
"#DB5A6B",
"#C93756",
"#FCC9B9",
"#FFB3A7",
"#F62459",
"#F58F84",
"#875F9A",
"#5D3F6A",
"#89729E",
"#763568",
"#8D608C",
"#A87CA0",
"#5B3256",
"#BF55EC",
"#8E44AD",
"#9B59B6",
"#BE90D4",
"#4D8FAC",
"#5D8CAE",
"#22A7F0",
"#19B5FE",
"#59ABE3",
"#48929B",
"#317589",
"#89C4F4",
"#4B77BE",
"#1F4788",
"#003171",
"#044F67",
"#264348",
"#7A942E",
"#8DB255",
"#5B8930",
"#6B9362",
"#407A52",
"#006442",
"#87D37C",
"#26A65B",
"#26C281",
"#049372",
"#2ABB9B",
"#16A085",
"#36D7B7",
"#03A678",
"#4DAF7C",
"#D9B611",
"#F3C13A",
"#F7CA18",
"#E2B13C",
"#A17917",
"#F5D76E",
"#F4D03F",
"#FFA400",
"#E08A1E",
"#FFB61E",
"#FAA945",
"#FFA631",
"#FFB94E",
"#E29C45",
"#F9690E",
"#CA6924",
"#F5AB35",
"#BFBFBF",
"#F2F1EF",
"#BDC3C7",
"#ECF0F1",
"#D2D7D3",
"#757D75",
"#EEEEEE",
"#ABB7B7",
"#6C7A89",
"#95A5A6"]
colors_count = len(colors)
colors_count

84

In [10]:
nodes = []
for node in NODES:
    nodes.append(
        new_node(
            color=colors[randint(0, colors_count-1)], 
            label=node, 
            attributes={}, 
            x=randint(-20, 20), 
            y=randint(-20, 20), 
            _id=node))
    
nodes

[{'attributes': {},
  'color': '#F4D03F',
  'id': 'AS45265',
  'label': 'AS45265',
  'size': 12,
  'x': 15,
  'y': -9},
 {'attributes': {},
  'color': '#407A52',
  'id': 'AS45642',
  'label': 'AS45642',
  'size': 17,
  'x': -19,
  'y': 20},
 {'attributes': {},
  'color': '#BF55EC',
  'id': 'AS63529',
  'label': 'AS63529',
  'size': 14,
  'x': 19,
  'y': 8},
 {'attributes': {},
  'color': '#E2B13C',
  'id': 'AS38081',
  'label': 'AS38081',
  'size': 18,
  'x': 19,
  'y': 13},
 {'attributes': {},
  'color': '#F62459',
  'id': 'AS133543',
  'label': 'AS133543',
  'size': 8,
  'x': -10,
  'y': -18},
 {'attributes': {},
  'color': '#F3C13A',
  'id': 'AS4652',
  'label': 'AS4652',
  'size': 37,
  'x': -7,
  'y': 13},
 {'attributes': {},
  'color': '#36D7B7',
  'id': 'AS45458',
  'label': 'AS45458',
  'size': 15,
  'x': 8,
  'y': -6},
 {'attributes': {},
  'color': '#DB5A6B',
  'id': 'AS37930',
  'label': 'AS37930',
  'size': 21,
  'x': 2,
  'y': -20},
 {'attributes': {},
  'color': '#1F4788'

In [11]:
edges = []
for item in EDGES:
    edges.append(new_edge(item[0], {}, item[1], 1))

edges

[{'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS45265'},
 {'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS38081'},
 {'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS45667'},
 {'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS45642'},
 {'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS45458'},
 {'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS132280'},
 {'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS9931'},
 {'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS4618'},
 {'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS4765'},
 {'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS7616'},
 {'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS4741'},
 {'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS45758'},
 {'attributes': {}, 'size': 1, 'sourceID': 'AS4652', 'targetID': 'AS24187'},
 {'

In [12]:
graph = {}
graph["nodes"] = nodes
graph["edges"] = edges

In [13]:
with open(FILE_NAME+".json", 'w') as outfile:
    json.dump(graph, outfile, indent=4, separators=(',', ': '), sort_keys=True)